## Compression Thoughts

TODO: Shuffle indexes once to simulate random access pattern then always index the same RGB bands and calculate the mean of each
to simulate _some_ computation measure the time it takes and then move the benchmark folder to a compressed folder!
I can check locally via BTRFS or squashfs or ...

But instinctively, I would assume that the block compression size should have to be aligned
with the size of the lmdb file 

-> Block compression is too inflexible and not configurable enough! (the blocks are too small to get good compressions)



In [1]:
# test size
import lmdb
import blosc2
from numpy import random
from safetensors.numpy import save
from pathlib import Path
from more_itertools import chunked
import rasterio

def read_single_band_raster(path: Path, index: int = 1):
    with rasterio.open(path) as r:
        return r.read(index)

example = Path("tiffs/BigEarthNet/BigEarthNet-S2/S2A_MSIL2A_20170613T101031_N9999_R022_T33UUP/S2A_MSIL2A_20170613T101031_N9999_R022_T33UUP_75_43/")
assert example.exists()
sample_data = {p.stem[-3:]: read_single_band_raster(p) for p in example.glob("*.tif")}
sample_data

{'B01': array([[ 397,  297,  323,  336,  358,  351,  351,  341,  405,  499,  508,
          839, 1145, 1314, 1237,  805,  242,  131,  148,  198],
        [ 332,  346,  378,  375,  379,  375,  373,  382,  465,  625,  620,
          852, 1157, 1301, 1153,  626,  167,  154,  128,  145],
        [ 361,  359,  331,  331,  304,  289,  317,  368,  484,  585,  647,
          791,  982, 1093,  835,  340,  144,  184,  179,  181],
        [ 332,  323,  259,  237,  239,  299,  387,  481,  575,  484,  409,
          424,  478,  540,  391,  176,  178,  231,  255,  308],
        [ 401,  355,  315,  291,  274,  600,  913,  980,  930,  560,  263,
          230,  209,  203,  172,  128,  175,  201,  208,  301],
        [ 548,  577,  557,  637,  619,  869, 1176, 1232, 1154,  619,  302,
          250,  162,  130,  156,  212,  399,  498,  359,  243],
        [ 896, 1014, 1075, 1056,  964, 1047, 1218, 1219, 1136,  701,  381,
          340,  236,  188,  211,  342,  669,  859,  716,  376],
        [ 705,  792,

In [2]:
# rand_band = lambda size: random.randint(2**15, size=(size, size)).astype("uint16")
# rand_s2 = lambda: {
#     "B01": rand_band(30),
#     "B02": rand_band(120),
#     "B03": rand_band(120),
#     "B04": rand_band(120),
#     "B05": rand_band(60),
#     "B06": rand_band(60),
#     "B07": rand_band(60),
#     "B08": rand_band(120),
#     "B8A": rand_band(60),
#     "B09": rand_band(60),
#     "B11": rand_band(60),
#     "B12": rand_band(30),
# }

In [9]:
sf_tns = save(sample_data)

c_d = blosc2.compress2(
    sf_tns,
    codec=blosc2.Codec.ZSTD,
    clevel=9,
    # filters=[blosc2.Filter.BYTEDELTA],
    filters=[blosc2.Filter.BITSHUFFLE],
    # filters=[blosc2.Filter.SHUFFLE],
)
# clevel 1: 0.7082980998806208
# 1.2sec/iteration for tqdm with clevel1 => Looks like scheduler issue to me. Just limit the number threads to maybe 2
len(c_d) / len(sf_tns)

0.7048970354158376

In [ ]:
# TODO: fix seed
env = lmdb.open("benchmark", map_size=2**40)
for chunk in chunked(range(10_000), 1_000):
    with env.begin(write=True) as txn:
        for i in chunk:
            txn.put(f"{i}".encode(), save(rand_s2()))